# 多进程
两种进程通信的方式：
- all-to-all
- master-worker: master就是个监工，worker是真正干活的人

In [111]:
using Distributed
using SharedArrays

In [17]:
addprocs(6)

6-element Vector{Int64}:
 2
 3
 4
 5
 6
 7

In [29]:
a = zeros(10)
@distributed for i = 1:10
    a[i] = Distributed.myid()
    @show a
end
@show a

a = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

      From worker 4:	a = [0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      From worker 3:	a = [0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      From worker 7:	a = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0]
      From worker 4:	a = [0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0]
      From worker 3:	a = [0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      From worker 6:	a = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0]
      From worker 2:	a = [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      From worker 2:	a = [2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      From worker 5:	a = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0]
      From worker 5:	a = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0]


In [32]:
@everywhere f1(x) = x*x
rst = @distributed (+) for i in 1:10
    f1(i)
end

385

# 多进程任务模型

In [104]:
r = @spawnat 3 Distributed.myid()

Future(3, 1, 312, nothing)

In [105]:
fetch(r)

3

In [102]:
t = @spawn Distributed.myid()
fetch(t)

7

In [34]:
s = remotecall(rand,3,2,2)

Future(3, 1, 177, nothing)

In [109]:
r = @time @spawnat 3 rand(10000,10000)
s = @time @spawnat 4 1 .+ fetch(r);
@time fetch(s);

  0.011540 seconds (283 allocations: 16.875 KiB)
  0.006893 seconds (320 allocations: 18.532 KiB)
 14.710260 seconds (606 allocations: 762.970 MiB, 29.05% gc time)


In [117]:
A = SharedArray{Float64}(1000,1000)
A .= rand(1000,1000)

rst = @time @distributed (+) for i in 1:1000
    sum(A, dims=1)
end

  0.378410 seconds (34.19 k allocations: 1.996 MiB, 14.50% compilation time)


1×1000 Matrix{Float64}:
 5.04928e5  497957.0  512307.0  505759.0  …  500796.0  4.9839e5  5.0837e5

In [120]:
rst = @time @distributed (+) for i in 1:1000
    A .= rand(1000,1)
    sum(A, dims=1)
end

  1.099442 seconds (478.52 k allocations: 29.743 MiB, 12.30% compilation time)


1×1000 Matrix{Float64}:
 5.00186e5  5.00165e5  500208.0  …  5.00207e5  5.00085e5  5.00075e5